In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, average_precision_score, f1_score, roc_curve, auc
from sklearn.metrics import precision_recall_curve
import seaborn as sns; sns.set()
sns.set_style('whitegrid')
sns.set_palette('Set2')
%matplotlib inline
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [3]:
def probability_to_label(probabilities, threshold=0.5):
    probabilities = list(probabilities)
    th = threshold
    predictions = [1 if i > th else 0 for i in probabilities]
    return predictions

In [4]:
def plot_cm(y_true, y_pred,
                          title='Confusion Matrix',
                          cmap=plt.cm.Blues):
    sns.set_style('white')
    
    cm = confusion_matrix(y_test, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # normarlize 
    
    from sklearn.utils.multiclass import unique_labels
    classes = unique_labels(y_true, y_pred)
    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' 
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    
    pass

In [5]:
def Imp_of_fea(name,weight):
    import operator
    import collections
    dic = dict(zip(name,weight))
    key_drop = [k for k in dic.keys() if  k == 'icustay_age_group']
    for k in key_drop:
        del dic[k]
    dic = sorted(dic.items(), key=operator.itemgetter(1))
    sorted_dict = collections.OrderedDict(dic)
    plt.style.use('seaborn-darkgrid')
    plt.figure(figsize=(5,20))
    plt.barh(list(sorted_dict.keys()),list(sorted_dict.values()))
    plt.title('Importance of features in 30 day mortality')

In [6]:
def plot_ROC(fpr, tpr, auc, title = 'ROC curve'):
    sns.set_style('whitegrid')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label=' (AUC = {:.3f})'.format(auc))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title(title)
    plt.legend(loc='best')
    plt.show()
    pass

In [7]:
def plot_PRC(precision, recall, ap, title= 'Precision-Recall Curve '):
    sns.set_style('whitegrid')

    plt.figure
#     plot(result5_1[0], result5_1[1],linestyle = "-", color = 'C0')#,label = 'AUC = %0.2f' % result5_1[3])
    plt.plot(recall, precision, lw=2, label='AP = %0.4f' % ap )
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(title)
    plt.legend(loc="best")
#     plt.show()
    return 

In [8]:
def get_weights(y):
    zeros = y[y==0].shape[0]
    ones = y[y==1].shape[0]
    weight = np.array([zeros, ones])*1.0/y.shape[0]
    y = np.where(y==0,weight[1],y)
    y = np.where(y==1,weight[0],y)
    return y

In [9]:
%run LoadData1Y.ipynb

Loaded 1 Year Train Sample:
X_train shape: (38857, 106)
y shape: (38857,)
Loaded 1 Year Test Sample:
X_test shape: (19139, 106)
y_test shape: (19139,)


In [10]:
X_train.head()

,first_careunit,last_careunit,age,gender,marital_status,insurance,urea_n_min,urea_n_max,urea_n_mean,platelets_min,...,meanbp_mean_mv,spo2_min_mv,spo2_max_mv,spo2_mean_mv,vent_mv,rrt_mv,urineoutput_mv,oasis_mv,lods_mv,sirs_mv
0,0.358226,-0.375558,-0.091698,-1.125040,0.002484,0.721077,0.336293,0.533833,0.568295,-0.307287,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.058760,1.040694,-0.138448,-1.125040,-0.795334,0.721077,1.458096,0.982380,1.364434,-0.617702,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.358226,-0.375558,4.139580,-1.125040,-0.795334,0.721077,-0.411576,-0.523458,-0.401365,-1.206421,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.058760,1.040694,0.115679,-1.125040,-0.795334,0.721077,0.261506,-0.395301,-0.145096,2.058292,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.358226,-0.375558,-0.268887,0.888858,0.800302,-0.508846,0.111932,-0.299184,-0.088050,-1.067269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
y_train.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: one_year, dtype: float64

In [12]:
param_grid = { 
    'learning_rate' : list(np.logspace(-5,1,5)),
    'n_estimators'  : list(np.linspace(60,150,10).astype(int))
}
model = GridSearchCV(
    estimator = XGBClassifier(verbosity=2),
    param_grid = param_grid,
    n_jobs = -1,
    verbose = 3,
    scoring = 'accuracy',
    cv = 5
).fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  8.5min finished


In [13]:
best_model = model.best_estimator_
ypred = best_model.predict_proba(X_test)
np.savetxt("plot metric/y_score_1y_xgboost.csv", ypred[:,1], delimiter=",")

In [14]:
best_model.score(X_test,y_test)

0.840378285176864

In [ ]:
xgacc = []
xgauc = []
for t_in,v_in in skf.split(X,y30):
    trainX = X.iloc[t_in].values
    trainy = y30.iloc[t_in]
    valX = X.iloc[v_in].values
    valy = y30.iloc[v_in]
    xg.fit(trainX,trainy.values.ravel(),get_weights(trainy.thirty_days))
    xgout = xg.predict_proba(valX)
#     xgout[xgout>0.5] = 1
#     xgout[xgout<=0.5] = 0
#     accuracy = accuracy_score(valy.values.ravel(), xgout)
#     print(accuracy)
    fpr, tpr, thresholds = roc_curve(valy.values.ravel(),xgout[:,1])
    roc_auc = auc(fpr, tpr)
    xgauc.append(roc_auc)
#     xgacc.append(accuracy_score(valy.values.ravel(), xgout))
    plt.figure()
    plt.plot(fpr,tpr)
    plt.show()

In [ ]:
def my_plot_importance(booster, figsize, **kwargs): 
    from matplotlib import pyplot as plt
    from xgboost import plot_importance
    fig, ax = plt.subplots(1,1,figsize=(figsize))
    plot_importance(booster=booster, ax=ax, **kwargs)
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label,] +
ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(10)
    plt.tight_layout()
    fig.savefig('figures/Feature_importance.png')

my_plot_importance(xg, (5,10))